In [3]:
import sys
import spotipy
import yaml
import spotipy.util as util
from pprint import pprint
import json
import argparse
import pandas as pd
import numpy as np

## Spotipy Credentials and API Calls

In [7]:
# Get Spotipy credentials from config
def load_config():
    stream = open('config.yaml')
    user_config = yaml.load(stream, Loader=yaml.FullLoader)
    return user_config

user_config = load_config()

In [14]:
# Initialize Spotify API token
token = util.prompt_for_user_token(user_config['username'],  client_id=user_config['client_id'], client_secret=user_config['client_secret'], redirect_uri=user_config['redirect_uri'])
sp = spotipy.Spotify(auth="BQBjqFngYfwuEdWGbyTEiLajD8UTeNqVWZfqHVOQiaG8ENpuwd_A-OBCqkoWLnOoiWoXdF7YGx00dJy7x8WwdWtNM3_RFP296kOjMNTAKqTxscz58IMJkP52r5eYijclCJnWNjfPj_SjNwRgcVL6Uy8kfxUicPISQIAu8UAmVXB68ng-7SNl4SgRu57Maxpxg7Hy4P7Gp3CwMp2fDY-uyro")

In [9]:
# A function to extract track names and URIs from a playlist
def get_playlist_info(username, playlist_uri):
    # initialize vars
    offset = 0
    tracks, uris, names, artists = [], [], [], []

    # get playlist id and name from URI
    playlist_id = playlist_uri.split(':')[2]
    
    playlist_name = sp.user_playlist(username, playlist_id)['name']

    # get all tracks in given playlist (max limit is 100 at a time --> use offset)
    while True:
        results = sp.user_playlist_tracks(username, playlist_id, offset=offset)
        tracks += results['items']
        if results['next'] is not None:
            offset += 100
            if(offset%100 == 0):
                print(offset, end=" ")
        else:
            break
        
    # get track metadata
    for track in tracks:
        try:
            names.append(track['track']['name'])
        except:
            pass
        try:
            artists.append(track['track']['artists'][0]['name'])
        except:
            pass
        try:
            uris.append(track['track']['uri'])
        except:
            pass
    
    return playlist_name, names, artists, uris

In [10]:
# Extract features from each track in a playlist
def get_features_for_playlist(df, names, artists, uris, playlist_name):
    print()
    i = 0
    # iterate through each track to get audio features and save data into dataframe
    for name, artist, track_uri in zip(names, artists, uris):
        if(i!=0  and i%100==0):
            print(i,end=" ")
        i+=1
        # print(json.dumps(track_uri, indent=4))              
        # ^ DEBUG STATEMENT ^
        
        try:
            # access audio features for given track URI via spotipy 
            audio_features = sp.audio_features(track_uri)

            # get relevant audio features
            feature_subset = [audio_features[0][col] for col in df.columns if col not in ["name", "artist", "track_URI", "playlist"]]

            # compose a row of the dataframe by flattening the list of audio features
            row = [name, artist, track_uri, *feature_subset, playlist_name]
            df.loc[len(df.index)] = row
        except:
            continue
    return df

In [15]:
# get all track metadata from given playlist
# Playlist 1 - top 10,000 songs - 'spotify:playlist:5S8SJdl1BDc0ugpkEvFsIL'
# Playlist 2 - dance party(best dance hits)- 400 songs - 5oKz4DsTP8zbL97UIPbqp4
# Playlist 3 - metal rock - 384 songs - 2xsIoqzZXdHDvnrot6Z8ql
# Playlist 4 - techno music - 149 songs - 18vUeZ9BdtMRNV6gI8RnR6
# Playlist 5 - acoustic - 460 songs - 1XqePqKyv638EBMhjpgYF9
# Playlist 6 - sad songs - 168 songs - 54NczufWOFmJ5jUVvRPxaF
# Playlist 7 - happy vibes - 110 songs - 7GhawGpb43Ctkq3PRP1fOL
# Playlist 8 - sleep - 338 songs - 7q4bsGZt7Uwkc3yZdu342B
# Playlist 9 - metal heavy - 200 songs - 37i9dQZF1DX9qNs32fujYe
# Playlist 10 - Low energy - 350 songs - 2ePGJPwhDvOp50h3DFij2N
playlist_name, names, artists, uris = get_playlist_info(user_config['username'], 'spotify:playlist:2ePGJPwhDvOp50h3DFij2N')

100 200 300 

In [4]:
# df = pd.DataFrame(columns=['name', 'artist', 'track_URI', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'playlist'])
df = pd.read_csv("data.csv")

In [9]:
# df = get_features_for_playlist(df, names, artists, uris, playlist_name)
df.head(1)

,name,artist,track_URI,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,playlist
0,...Ready For It?,Taylor Swift,spotify:track:7zgqtptZvhf8GEmdsM2vp2,0.0665,0.615,0.779,0.0,0.155,-6.454,0.135,160.0,0.453,The Longest Playlist on Spotify® (Official)


In [18]:
print(df.shape)
print(df.duplicated(keep=False).sum())
df = df.drop_duplicates(subset='track_URI', keep='first')
print(df.shape)

(12178, 13)
0
(12138, 13)


In [5]:
df.to_csv("data.csv", index=False)